# Admin interface 

Django provides a helpful admin interface to work with objects in it.

# Registering models

You don't have any models in the admin interface as a start. To add one, register them in the admin.py module. This module exists in each application by default.

Let's look at a hypothetical project. We are creating an online auction but do not have much time to prepare the first release. To avoid using external dependencies, we change the site's content in the Django admin interface. The models look like this:


In [ ]:
from django.contrib.auth.models import User
from django.db import models


class Lot(models.Model):
    description = models.CharField(max_length=256)
    initial_price = models.FloatField()


class Bid(models.Model):
    lot = models.ForeignKey(Lot, on_delete=models.CASCADE)
    price = models.FloatField()
    user = models.ForeignKey(User, on_delete=models.CASCADE)

To see models in the admin interface, register them with the register function. Let's see what we should add to the ```admin.py``` module.

In [ ]:
from django.contrib import admin
from .models import Bid, Lot


admin.site.register(Bid)
admin.site.register(Lot)

# Customize fields

Not all data should be open for changes. For example, our clients won't be happy if we adjust their bids. No matter how honest developers and business users are, it's better not to let them change any data in the Bid objects. To customize fields, use the ModelAdmin class.

The instance of the ModelAdmin class has a lot of customizable attributes. The fields attribute is the list of model fields you want to display on a page. The readonly_fields is the list of fields that all admins can see but not modify. Both attributes can be tuple or list instances.

In [ ]:
class BidAdmin(admin.ModelAdmin):
    fields = ('lot', 'price', 'user')
    readonly_fields = ('lot', 'price', 'user')

admin.site.register(Bid, BidAdmin)

# Additional fields

It's great to get the plain data from the tables with several lines of code, but how do we add more information from the database?

We want to see which lots are trending. In our case, it's the lots with ten bids or more. There is no such field in a model, but we can prepare the ModelAdmin field with this value. Choose any name you want unless it intersects with the existing attribute of the class. For more info, take a look at the attributes in the documentation. We've added the trending method with the string trending to the fields list:

In [ ]:
class LotAdmin(admin.ModelAdmin):
    fields = ('description', 'initial_price', 'trending')
    readonly_fields = ('trending',)

    def trending(self, obj):
        return '🔥' if obj.bid_set.count() >= 10 else '❄'

Do not forget to add a custom field to the readonly_fields list; otherwise, it will produce an error. Custom fields are not modifiable.

# Inline admin field

We can add any fields we like if they are custom values or parts of a model. We can also use the inline fields to display all the bids for a lot on the page.

This time, create an additional class for displaying each inline object. You want to add bids to a page with the lot, so create the TabularInline class for the Bid model:

In [ ]:
from django.contrib import admin
from .models import Bid, Lot


class BidInline(admin.TabularInline):
    can_delete = False
    model = Bid
    readonly_fields = ('price', 'user')


class LotAdmin(admin.ModelAdmin):
    inlines = (BidInline,)


admin.site.register(Lot, LotAdmin)

We don't register the BidInline class because we cannot use it independently, but we define the model attribute for it. Restrict the actions for this class with read-only fields and set can_delete attribute to False to prevent deleting bids by the admin user.

In the LotAdmin class, define the inlines attribute and add the BidInline to it.

That's it; all bids with the lot item are displayed on the same page.

# Sorting and ordering

Our app became popular, and we now have hundreds of lots and bids. When administrators open their list, it's easy for them to get confused, and finding the right lot or bid is tough. To fix the situation, you can use the sorting of available items.

In [ ]:
from django.contrib import admin
from .models import Bid

class BidAdmin(admin.ModelAdmin):
    list_display = ('lot', 'price', 'user')
    ordering = ('-price',)

admin.site.register(Bid, BidAdmin)